In [8]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np


df = pd.read_csv("/content/ner.csv")

import ast

sentences = np.array(df['Sentence'])
tags = df['Tag']

# Fill NaN values in the 'tags' column with an empty list string before applying literal_eval
parsed_tags = tags.fillna('[]').apply(ast.literal_eval)

for i in range(len(sentences)):
    sentences[i] = sentences[i].lower()
print(sentences[-1])

x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(sentences)
x_data = x_tokenizer.texts_to_sequences(sentences)
X = pad_sequences(x_data, padding='post')
max_x_seq = X.shape[1]
# Determine the maximum sequence length from X to ensure Y matches
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(parsed_tags) # Use parsed_tags here
y_data = y_tokenizer.texts_to_sequences(parsed_tags) # Use parsed_tags here
Y = pad_sequences(y_data, padding='post',maxlen=max_x_seq) # Pad Y to match X's sequence length
x_train , x_test , y_train, y_test = train_test_split(X,Y, test_size= 0.2)




indian forces said they responded to the attack


In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense

input_layer = Input(shape=(max_x_seq,))
embedding = Embedding(
    input_dim=(len(x_tokenizer.word_index)+1),
    output_dim=128,
)(input_layer)

bilstm = Bidirectional(
    LSTM(64, return_sequences=True)
)(embedding)

output = Dense((len(y_tokenizer.word_index)+1), activation="softmax")(bilstm)

model = Model(input_layer, output)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

model.fit(
    x_train,
   y_train,
   validation_data = (x_test,y_test),
    epochs=1
)



Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 89)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, 89, 128)        │     3,578,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 89, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 89, 18)         │         2,322 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,679,250 (14.04 MB)

 Trainable params: 3,679,250 (14.04 MB)

 Non-trainable params: 0 (0.00 B)

1199/1199 ━━━━━━━━━━━━━━━━━━━━ 189s 155ms/step - accuracy: 0.9367 - loss: 0.2581 - val_accuracy: 0.9645 - val_loss: 0.1093


In [10]:
import pickle

with open('x_tokenizer.pkl', 'wb') as f:
    pickle.dump(x_tokenizer, f)

with open('y_tokenizer.pkl', 'wb') as f:
    pickle.dump(y_tokenizer, f)

model.save('ner_model.h5')